In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]=" "

# Load Network 15AB

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_1/models/B5R2_block5_15AB_1FC_3.h5' #model
# model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_Full/models/B5R2_15AB_Full_1.h5'
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image/'#แก้
dataframe = pd.read_csv( '/home/yupaporn/CCA_Project/Dataset_CCA_case_Testset1312.csv')# ---------------------------------------*เปลี่ยนไฟล์เเล้ว
validation_dir = os.path.join(base_dir, 'test')

#Train
train_df = pd.read_csv( '/home/yupaporn/CCA_Project/Dataset_CCA_case_Trainset5257.csv')# ---------------------------------------*เปลี่ยนไฟล์เเล้ว
base_dir0 = '/media/tohn/SSD/Images/Image/'#แก้
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
dataframe = pd.read_csv ('/home/yupaporn/CSV_file/UICCA_DiagRadioExp_AzureDb51Case813Image.csv')# ---------------------------------------*เปลี่ยนไฟล์เเล้ว
print(dataframe .shape)
# dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# print(dataframe.shape)
# print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
# print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(813, 25)


,Unnamed: 0,Case,FileName,Views,Label,Width,Height,X1,Y1,X2,...,date,originalImage,Path Full,Class,Sub_class_name,Sub_class,Path Crop,Path Crop_I7,Sub_Class_15AB,Views_
0,0,12215,12215_9.jpg,P42,Normal,800,600,219.60,113.40,518.50,...,20/10/2021,1.2.392.200039.110.1.109018960.10.20211020.150...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort/12...,Normal,['Normal'],['Normal'],/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,FP-B
1,1,12215,12215_8.jpg,P41,Normal,800,600,227.00,120.20,553.10,...,20/10/2021,1.2.392.200039.110.1.109018960.10.20211020.150...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort/12...,Normal,['Normal'],['Normal'],/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,FP-B
2,2,12215,12215_7.jpg,P52,Normal,800,600,290.00,129.30,481.10,...,20/10/2021,1.2.392.200039.110.1.109018960.10.20211020.150...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort/12...,Normal,['Normal'],['Normal'],/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,FP-C
3,3,12215,12215_6.jpg,P52,Normal,800,600,289.90,138.40,547.43,...,20/10/2021,1.2.392.200039.110.1.109018960.10.20211020.150...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort/12...,Normal,['Normal'],['Normal'],/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,FP-C
4,4,12215,12215_5.jpg,P61,Normal,800,600,236.70,132.70,542.30,...,20/10/2021,1.2.392.200039.110.1.109018960.10.20211020.150...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort/12...,Normal,['Normal'],['Normal'],/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,FP-C


In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Sub_Class_15AB',# ---------------------------------------*เปลี่ยนไฟล์เเล้ว
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 813 validated image filenames belonging to 11 classes.
{0: 'AB01', 1: 'AB02', 2: 'AB03', 3: 'AB05', 4: 'AB06', 5: 'AB081', 6: 'AB09', 7: 'AB10', 8: 'AB11', 9: 'None', 10: 'Normal'}


In [7]:
labels = {0: 'AB01', 1: 'AB02', 2: 'AB03', 3: 'AB04', 4: 'AB05', 5: 'AB06', 6: 'AB07', 7: 'AB081', 8: 'AB082', 9: 'AB083', 10: 'AB09', 11: 'AB10', 12: 'AB11', 13: 'AB12', 14: 'Normal'}

In [8]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category_BiTNet'] = pred_list
dataframe['Prob'] = prob_list

In [9]:
vector_array = np.array(vectorlist)
vector_array

array([[ 2.03359187e-01,  1.99759528e-01, -1.84847996e-01, ...,
        -5.73811005e-04, -1.95445251e-02,  1.67188197e-01],
       [-4.08402011e-02,  2.62666374e-01, -1.53788790e-01, ...,
        -7.65286088e-02,  1.69447899e-01,  8.55686441e-02],
       [-1.24315044e-03,  1.82002690e-02, -1.25296220e-01, ...,
         7.44693503e-02, -2.04598792e-02, -7.47057498e-02],
       ...,
       [ 2.76687771e-01,  2.88524956e-01, -1.16330251e-01, ...,
         4.74685758e-01, -2.71723680e-02,  1.03506595e-01],
       [-2.40908153e-02,  3.21784094e-02, -5.27889170e-02, ...,
         1.09737292e-01,  1.76945910e-01,  2.77735144e-01],
       [ 7.02583119e-02,  6.13981545e-01, -1.44177347e-01, ...,
         1.33283019e-01,  1.50299683e-01,  3.80106181e-01]])

In [10]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.203359,0.199760,-0.184848,-0.098532,-0.013952,0.472806,-0.116408,0.007638,0.059848,-0.124445,...,-0.066696,-0.020666,0.270880,0.047198,-0.169408,-0.117276,-0.148420,-0.000574,-0.019545,0.167188
1,-0.040840,0.262666,-0.153789,-0.099390,-0.177379,-0.064506,-0.068917,-0.213335,0.529789,-0.075279,...,-0.119028,0.008125,0.237100,0.177209,-0.026377,-0.183636,-0.021608,-0.076529,0.169448,0.085569
2,-0.001243,0.018200,-0.125296,-0.122112,-0.181850,-0.081947,-0.152434,-0.224691,-0.160313,-0.170663,...,-0.152352,0.153477,-0.172108,0.374136,0.041474,-0.125647,-0.077779,0.074469,-0.020460,-0.074706
3,0.048955,0.135096,-0.152800,-0.184129,-0.202729,0.096376,-0.144791,-0.202041,-0.127833,-0.188167,...,-0.127660,-0.094425,0.025626,0.746567,0.017456,-0.135341,-0.127499,0.302009,0.039836,0.008306
4,-0.153857,0.194617,-0.149463,-0.161262,-0.201524,-0.151863,-0.165302,-0.123635,0.128445,-0.081327,...,-0.018674,0.222626,0.003855,0.369354,0.108896,-0.188593,-0.109908,-0.036442,-0.096678,-0.024667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808,0.589089,1.023898,-0.154509,-0.116566,0.550815,0.209012,-0.051862,0.125612,-0.029306,0.016580,...,-0.090554,-0.069812,0.815583,0.497383,0.397092,-0.111215,0.303707,1.009950,0.061688,0.727066
809,0.047955,0.225766,-0.179531,-0.126144,0.202625,0.331322,-0.052787,0.070805,0.542371,0.038342,...,0.006419,0.015993,0.276189,0.702364,-0.016808,0.337929,0.039312,0.179155,-0.025846,0.499964
810,0.276688,0.288525,-0.116330,-0.079801,-0.061018,-0.070026,-0.071984,0.189211,0.279064,0.012907,...,-0.135275,0.077640,0.422160,0.560112,0.037872,-0.076958,-0.137947,0.474686,-0.027172,0.103507
811,-0.024091,0.032178,-0.052789,-0.125494,0.172052,-0.001986,-0.058659,-0.168698,-0.061381,-0.186612,...,-0.148947,-0.048695,-0.187583,0.805689,0.141571,-0.002560,-0.170878,0.109737,0.176946,0.277735


In [16]:
dataframe

,Unnamed: 0,Case,FileName,Views,Label,Width,Height,X1,Y1,X2,...,Path Full,Class,Sub_class_name,Sub_class,Path Crop,Path Crop_I7,Sub_Class_15AB,Views_,category_BiTNet,Prob
0,0,12215,12215_9.jpg,P42,Normal,800,600,219.60,113.40,518.50,...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort/12...,Normal,['Normal'],['Normal'],/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,FP-B,Normal,0.947677
1,1,12215,12215_8.jpg,P41,Normal,800,600,227.00,120.20,553.10,...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort/12...,Normal,['Normal'],['Normal'],/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,FP-B,Normal,0.728153
2,2,12215,12215_7.jpg,P52,Normal,800,600,290.00,129.30,481.10,...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort/12...,Normal,['Normal'],['Normal'],/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,FP-C,AB12,0.999998
3,3,12215,12215_6.jpg,P52,Normal,800,600,289.90,138.40,547.43,...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort/12...,Normal,['Normal'],['Normal'],/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,FP-C,AB12,0.941997
4,4,12215,12215_5.jpg,P61,Normal,800,600,236.70,132.70,542.30,...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort/12...,Normal,['Normal'],['Normal'],/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,FP-C,AB12,0.999980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808,808,12150,12150_13.jpg,P42,Normal,800,600,215.10,108.90,510.00,...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort/12...,Normal,['Normal'],['Normal'],/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,FP-B,Normal,1.000000
809,809,12150,12150_12.jpg,P42,Normal,800,600,194.69,106.70,526.40,...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort/12...,Normal,['Normal'],['Normal'],/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,FP-B,Normal,1.000000
810,810,12150,12150_11.jpg,P52,Normal,800,600,249.10,114.60,597.90,...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort/12...,Normal,['Normal'],['Normal'],/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,FP-C,Normal,0.999914
811,811,12150,12150_10.jpg,P52,Normal,800,600,270.70,135.00,574.10,...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort/12...,Normal,['Normal'],['Normal'],/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,FP-C,Normal,0.937677


In [17]:
b = dataframe['FileName'].tolist()
b2 = dataframe['Case'].tolist()
b3 = dataframe['Path Crop'].tolist()
b4 = dataframe['Class'].tolist()
b5 = dataframe['Sub_Class_15AB'].tolist()
b6 = dataframe['Views_'].tolist()
b7 = dataframe['category_BiTNet'].tolist()
b8 = dataframe['Prob'].tolist()

In [18]:
dataset['FileName'] = b
dataset['Case'] = b2
dataset['Path crop'] = b3
dataset['Class'] = b4
dataset['Sub_Class_15AB'] = b5
dataset['Views_'] = b6
dataset['category_BiTNet'] = b7
dataset['Prob'] = b8

In [19]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2046,2047,FileName,Case,Path crop,Class,Sub_Class_15AB,Views_,category_BiTNet,Prob
0,0.203359,0.199760,-0.184848,-0.098532,-0.013952,0.472806,-0.116408,0.007638,0.059848,-0.124445,...,-0.019545,0.167188,12215_9.jpg,12215,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,Normal,FP-B,Normal,0.947677
1,-0.040840,0.262666,-0.153789,-0.099390,-0.177379,-0.064506,-0.068917,-0.213335,0.529789,-0.075279,...,0.169448,0.085569,12215_8.jpg,12215,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,Normal,FP-B,Normal,0.728153
2,-0.001243,0.018200,-0.125296,-0.122112,-0.181850,-0.081947,-0.152434,-0.224691,-0.160313,-0.170663,...,-0.020460,-0.074706,12215_7.jpg,12215,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,Normal,FP-C,AB12,0.999998
3,0.048955,0.135096,-0.152800,-0.184129,-0.202729,0.096376,-0.144791,-0.202041,-0.127833,-0.188167,...,0.039836,0.008306,12215_6.jpg,12215,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,Normal,FP-C,AB12,0.941997
4,-0.153857,0.194617,-0.149463,-0.161262,-0.201524,-0.151863,-0.165302,-0.123635,0.128445,-0.081327,...,-0.096678,-0.024667,12215_5.jpg,12215,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,Normal,FP-C,AB12,0.999980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808,0.589089,1.023898,-0.154509,-0.116566,0.550815,0.209012,-0.051862,0.125612,-0.029306,0.016580,...,0.061688,0.727066,12150_13.jpg,12150,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,Normal,FP-B,Normal,1.000000
809,0.047955,0.225766,-0.179531,-0.126144,0.202625,0.331322,-0.052787,0.070805,0.542371,0.038342,...,-0.025846,0.499964,12150_12.jpg,12150,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,Normal,FP-B,Normal,1.000000
810,0.276688,0.288525,-0.116330,-0.079801,-0.061018,-0.070026,-0.071984,0.189211,0.279064,0.012907,...,-0.027172,0.103507,12150_11.jpg,12150,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,Normal,FP-C,Normal,0.999914
811,-0.024091,0.032178,-0.052789,-0.125494,0.172052,-0.001986,-0.058659,-0.168698,-0.061381,-0.186612,...,0.176946,0.277735,12150_10.jpg,12150,/media/tohn/SSD/ImageFromAzureDB_IsanCohort_cr...,Normal,Normal,FP-C,Normal,0.937677


In [15]:
set(dataset[dataset['Class'] == 'Normal']['Sub_class_name'])

{"['Normal']"}

In [20]:
os.chdir( '/home/yupaporn/CCA_Project')
testdf = dataset
# testdf.to_csv('FeatureVector_CCA_case_DiagRadioExp_AzureDb813_modelFull.csv') 